In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
database_1 = pd.read_csv('/Users/irfanhilman/Downloads/Database/Database Ringkas/Database Part 1.csv')
database_2 = pd.read_csv('/Users/irfanhilman/Downloads/Database/Database Ringkas/Database Part 2.csv')
database_3 = pd.read_csv('/Users/irfanhilman/Downloads/Database/Database Ringkas/Database Part 3.csv')
database_4 = pd.read_csv('/Users/irfanhilman/Downloads/Database/Database Ringkas/Database Part 4.csv')
database = pd.concat([database_1,database_2,database_3,database_4])

In [3]:
one_month = 21
rolling_dic = {'1 Month':one_month,'3 Month':one_month*3,'6 Month':one_month*6,'1 Year':one_month*12}
def rolling_average(df,param,period):
    init_list = ['Kode','Date']
    init_list.append(param)
    df['Kode + Date'] = df['Kode'] + df['Date'].astype('str')
    df_ = df[init_list]
    list_df = []
    for i in set(list(df_['Kode'])):
        df_i = df_[df_['Kode']==i]
        df_i[f'Average {param} {period}'] = df_i[f'{param}'].rolling(rolling_dic[period],min_periods=1).mean() # Min 1 data point
        list_df.append(df_i)
    df__ = pd.concat(list_df).sort_values(by=['Kode','Date'])
    df__['Kode + Date'] = df__['Kode'].astype('str') + df__['Date'].astype('str')
    df__ = df__.drop_duplicates(subset='Kode + Date')
    df = df.merge(df__[[f'Average {param} {period}','Kode + Date']],on='Kode + Date',how='left')
    return df

In [4]:
database = rolling_average(df=database,param='Transaction Value',period='3 Month')

/var/folders/rj/b4yv8pv54v31_6jpqn2byxgc0000gn/T/ipykernel_80961/164283080.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_i[f'Average {param} {period}'] = df_i[f'{param}'].rolling(rolling_dic[period],min_periods=1).mean() # Min 1 data point
/var/folders/rj/b4yv8pv54v31_6jpqn2byxgc0000gn/T/ipykernel_80961/164283080.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_i[f'Average {param} {period}'] = df_i[f'{param}'].rolling(rolling_dic[period],min_periods=1).mean() # Min 1 data point
/var/folders

In [5]:
universe = pd.read_excel('Konstituen Pegadaian.xlsx')
universe = universe.merge(database,on='Kode',how='left')

In [6]:
reb_date = ['2025-01-02','2024-09-02','2024-06-03','2024-03-01','2024-01-02','2023-09-01','2023-06-05','2023-03-01','2023-01-02',
            '2022-09-01','2022-06-02','2022-03-01','2022-01-04','2021-09-01','2021-06-02','2021-03-01','2021-01-04','2020-09-01',
            '2020-06-02','2020-03-02','2020-01-02','2019-09-02','2019-06-10','2019-03-01','2019-01-02']

In [7]:
drop_list = ['VISI','BDKR','GEMS','BSML','MKAP','NEST','GULA','NICE']

In [8]:
with pd.ExcelWriter(f'Universe Saham Pegadaian Filter 3 Bulan.xlsx') as writer:
    for i in reb_date:
        universe_i = universe[universe['Date']==i]
        universe_i = universe_i[universe_i['Average Transaction Value 3 Month']>5_000_000_000]
        universe_i = universe_i[universe_i['FFMC']>200_000_000_000]
        universe_i = universe_i[universe_i['Close Price']>50]
        universe_i = universe_i[universe_i['Ekuitas']>0]
        if datetime.strptime(i, '%Y-%m-%d')>datetime.strptime('2020-03-31', '%Y-%m-%d'):
            universe_i = universe_i[universe_i['Average Frequency 1 Month']>500]
        else: pass
        universe_i = universe_i[universe_i['Distance to First Date'].str[:-5].astype('float64').fillna(0) > 120]
        for j in drop_list:
            universe_i = universe_i[universe_i['Kode']!=j]
        universe_i = universe_i[['Kode','Date','Quarter Year','Distance to First Date','Close Price','Average Transaction Value 3 Month','Average Transaction Value 1 Month','Frequency','Average Frequency 1 Month','Debt Ratio','Ekuitas','Transaction Value/Frequency',
                            'Average Market Cap 1 Month','Free Float','FFMC']]
        universe_i = universe_i.sort_values(by='FFMC',ascending=False)
        universe_i = universe_i.head(100) #Ambil 100 saham
        universe_i['Weight'] = universe_i['FFMC']/sum(universe_i['FFMC'])
        universe_i = universe_i.reset_index(drop=True)
        universe_i.to_excel(writer, sheet_name=f'{i}')